<img src="https://upload.wikimedia.org/wikipedia/commons/7/76/Grain_logo.png" style="float: left; margin: 0px; height: 80px">

<div style="clear: both;"></div>

<div>
    <h1>Capstone project: Business dashboard</h1>
    <p>Part 1 - Data cleaning</p>
</div>

---
## Problem Statement
The F&B industry in Singapore is valued at $13 billion a year. The food catering industry specifically is picking up post-covid19 and is expected to grow in the next few years. However, there are many issues that catering companies have to solve. We will focus on 3 issues.

1. Inventory management (Given past data, can we forecast demand to efficiently carry out inventory management?)
2. Price negotiation (What is the recommended price for customers from different sectors?)
3. Food quality: Delivery driver allocation (Is there a way to automatically assign drivers for deliveries to pick up hot food at the right timings and to effectively distribute the deliveries so that only a final check is needed?)

## Contents:
- [Import libraries](#Import-libraries) 
- [Import data](#Import-data)
- [Data cleaning](#Data-cleaning)
- [Data imputation](#Data-imputation)

# Import libraries 

In [1]:
import pandas as pd

# Import data 

In [2]:
# Import data from data file
# This is 1 year worth of orders for Minibuffet
df = pd.read_csv("../data/Mini_buffets_2022.csv")

# Set the view limit to none
pd.set_option('display.max_columns', None) 
print(df.shape)
df.head()

(11475, 40)


,event_id,order_id,event_state,sales_order_state,sales_type,trial,promo_code,referred_by,sector,company_name,customer_id,type,sub_type,cost_per_pax,lines,ways,pax,address,postal_code,need_to_use_stairs,special_instructions,need_pre_setup,pre_setup_date_time,created_at,updated_at,pickup_date_time,setup_date_time,serving_date_time,takedown_date_time,ready_to_eat_at,consume_by,sub_total_value,additional_charges_value,delivery_fee,surcharge_value,discount_value,revenue,event_url,sales_order_url,invoice_url
0,E8162,SO070833480,PUBLISHED,CONFIRMED,inbound,False,NaN,NaN,NaN,NaN,45602,MiniBuffet,NaN,18.00,1,1,15,"120 King's Rd, Church of St. Ignatius",268172,NaN,Customised label: Thank you for making this sp...,False,NaN,2021-07-08T12:25:35.03374,2022-02-24T16:17:14.542858,2022-02-26T10:45:00,2022-02-26T11:45:00,2022-02-26T11:45:00,2022-02-26T11:45:00,2022-02-26T10:45:00,2022-02-26T14:45:00,270.0,77.85,30.0,0.0,0.00,377.85,https://papercut.grain.com.sg/events/58162,https://papercut.grain.com.sg/sales_orders/33480,https://go.xero.com/AccountsReceivable/Edit.as...
1,F1912,SO120836198,PUBLISHED,CONFIRMED,outbound,False,NaN,NaN,Hospitals,National University Hospital (S) Pte Ltd,31632,MiniBuffet,NaN,12.99,1,1,160,"1E Kent Ridge Road, NUHS Tower Block, Level 8",119228,NaN,160 bento & cutlery | 160 fruit parcels | 250 ...,False,NaN,2021-12-08T21:50:13.615565,2022-02-21T17:15:50.026876,2022-02-23T09:30:00,2022-02-23T10:30:00,2022-02-23T10:30:00,2022-02-23T10:30:00,2022-02-23T09:30:00,2022-02-23T13:30:00,2078.4,0.00,15.0,0.0,15.00,2078.40,https://papercut.grain.com.sg/events/61912,https://papercut.grain.com.sg/sales_orders/36198,https://go.xero.com/AccountsReceivable/Edit.as...
2,F1914,SO120836198,PUBLISHED,CONFIRMED,outbound,False,NaN,NaN,Hospitals,National University Hospital (S) Pte Ltd,31632,MiniBuffet,NaN,12.99,1,1,90,"1E Kent Ridge Road, NUHS Tower Block, Level 8",119228,NaN,90 bento & cutlery | 90 fruit parcels,False,NaN,2021-12-08T22:02:35.998535,2022-02-21T17:15:51.700405,2022-02-23T09:30:00,2022-02-23T10:30:00,2022-02-23T10:30:00,2022-02-23T10:30:00,2022-02-23T09:30:00,2022-02-23T13:30:00,1169.1,0.00,15.0,0.0,15.00,1169.10,https://papercut.grain.com.sg/events/61914,https://papercut.grain.com.sg/sales_orders/36198,https://go.xero.com/AccountsReceivable/Edit.as...
3,F2598,SO121236841,PUBLISHED,CONFIRMED,inbound,False,RF991822,NaN,NaN,NaN,49671,MiniBuffet,NaN,33.60,1,1,5,"123 Teck Whye Lane, #06-850",680123,NaN,Please hand over the delivery to a household m...,False,NaN,2021-12-12T10:25:59.331641,2022-01-06T13:55:04.219927,2022-01-08T18:00:00,2022-01-08T19:00:00,2022-01-08T19:00:00,2022-01-08T19:00:00,2022-01-08T18:00:00,2022-01-08T22:00:00,168.0,44.81,30.0,0.0,12.14,230.67,https://papercut.grain.com.sg/events/62598,https://papercut.grain.com.sg/sales_orders/36841,NaN
4,F2672,SO121236916,PUBLISHED,CONFIRMED,repeat,False,NaN,NaN,NaN,NaN,47363,MiniBuffet,NaN,18.00,1,1,126,"Amazing Grace Presbyterian Church, 408B Upper ...",466484,NaN,252 x Bento + Cutlery \n\n130 x paper bags\n\n...,False,NaN,2021-12-12T23:30:49.300378,2022-01-06T13:55:10.589043,2022-01-08T10:30:00,2022-01-08T11:30:00,2022-01-08T11:30:00,2022-01-08T11:30:00,2022-01-08T10:30:00,2022-01-08T14:30:00,2268.0,65.00,0.0,0.0,0.00,2333.00,https://papercut.grain.com.sg/events/62672,https://papercut.grain.com.sg/sales_orders/36916,https://go.xero.com/AccountsReceivable/Edit.as...


# Data cleaning

**Remove the rows where the event_state is DELETED as a customer may have placed an order but later changed their mind and cancelled it**

In [3]:
# Filter out rows where 'event_state' is not equal to 'DELETED'
df = df[df['event_state'] != 'DELETED']
df.shape

(11121, 40)

There were initially 11, 475 rows and after removing the deleted events, we are left with 11, 121 rows. This means that there were 354 deleted events.

**Choose the columns that we want**

In [4]:
# Check if the event ID is unique by adding up the sum of all the unique IDs
num_of_events = df['event_id'].nunique()
num_of_events

11121

Yes the event ID's are unique since the number of unique event ID's is equal to the number of rows in the dataframe

In [5]:
# Reorganise the dataframe
new_df = df[['event_id', 'event_state', 'sales_type', 'sector', 'company_name', 'customer_id', 'cost_per_pax', 'pax', 'address', 'postal_code', 'special_instructions', 'pickup_date_time', 'serving_date_time', 'revenue']]
print(new_df.shape)
new_df.head(60)

(11121, 14)


,event_id,event_state,sales_type,sector,company_name,customer_id,cost_per_pax,pax,address,postal_code,special_instructions,pickup_date_time,serving_date_time,revenue
0,E8162,PUBLISHED,inbound,NaN,NaN,45602,18.00,15,"120 King's Rd, Church of St. Ignatius",268172,Customised label: Thank you for making this sp...,2022-02-26T10:45:00,2022-02-26T11:45:00,377.85
1,F1912,PUBLISHED,outbound,Hospitals,National University Hospital (S) Pte Ltd,31632,12.99,160,"1E Kent Ridge Road, NUHS Tower Block, Level 8",119228,160 bento & cutlery | 160 fruit parcels | 250 ...,2022-02-23T09:30:00,2022-02-23T10:30:00,2078.40
2,F1914,PUBLISHED,outbound,Hospitals,National University Hospital (S) Pte Ltd,31632,12.99,90,"1E Kent Ridge Road, NUHS Tower Block, Level 8",119228,90 bento & cutlery | 90 fruit parcels,2022-02-23T09:30:00,2022-02-23T10:30:00,1169.10
3,F2598,PUBLISHED,inbound,NaN,NaN,49671,33.60,5,"123 Teck Whye Lane, #06-850",680123,Please hand over the delivery to a household m...,2022-01-08T18:00:00,2022-01-08T19:00:00,230.67
4,F2672,PUBLISHED,repeat,NaN,NaN,47363,18.00,126,"Amazing Grace Presbyterian Church, 408B Upper ...",466484,252 x Bento + Cutlery \n\n130 x paper bags\n\n...,2022-01-08T10:30:00,2022-01-08T11:30:00,2333.00
5,F2830,PUBLISHED,repeat,"Oil & Gas, Chemicals and Energy",Coim Asia Pacific Pte Ltd,2230,16.73,12,10 Seraya Place,627843,24 x meal box + cutleries,2022-01-11T11:00:00,2022-01-11T12:00:00,230.76
6,F3176,PUBLISHED,inbound,NaN,NaN,50203,0.00,10,"910 Hougang Street 91, #06-80",530910,Morning Bundle B and E\n23 x Bentos and Cutleries,2022-01-08T07:30:00,2022-01-08T08:30:00,428.85
7,F3177,PUBLISHED,inbound,NaN,NaN,46587,16.73,18,Block 432 Clementi Ave 3 #06-294,120432,Please take the single lift lobby (beside Poon...,2022-01-08T12:00:00,2022-01-08T13:00:00,321.79
8,F3201,PUBLISHED,repeat,Pharmaceutical,Novartis (Singapore) Pte Ltd,26299,7.43,30,"1E Kent Ridge Rd, National University Cancer I...",119228,"30 x Bento + Cutlery | 39 x ripple cups, lids,...",2022-01-06T07:15:00,2022-01-06T08:15:00,308.13
9,F3202,PUBLISHED,repeat,Pharmaceutical,Novartis (Singapore) Pte Ltd,26299,7.43,30,"1E Kent Ridge Rd, National University Cancer I...",119228,"30 x Bento + Cutlery | 39 x ripple cups, lids,...",2022-01-13T07:15:00,2022-01-13T08:15:00,308.13


We chose 14 relevant columns from the 40 that were in the dataset.

**Check for duplicate rows**

In [6]:
# Assuming your DataFrame is called 'df'
num_duplicate_rows = new_df.duplicated().sum()
num_duplicate_rows

0

There are no duplicated rows. 

**Check for null values**

In [7]:
# Get the data types by column
data_types_by_column = new_df.dtypes

# Get the number of null values by column
null_values_by_column = new_df.isnull().sum()

# Get the percentage of null values by column
percentage_of_null_values = round((new_df.isnull().sum() / len(new_df)) * 100)

# Combine the data types and null values into a DataFrame
data_info = pd.DataFrame({'Data Type': data_types_by_column, 'Null Values': null_values_by_column, 'Percentage of null values(%)': percentage_of_null_values})

# Print the result
data_info

,Data Type,Null Values,Percentage of null values(%)
event_id,object,0,0.0
event_state,object,0,0.0
sales_type,object,0,0.0
sector,object,4453,40.0
company_name,object,2596,23.0
customer_id,int64,0,0.0
cost_per_pax,float64,0,0.0
pax,int64,0,0.0
address,object,0,0.0
postal_code,int64,0,0.0


There are quite a lot of missing values for the sector, company_name and special_instructions. We will impute the missing values in the next segement.

# Data imputation

**Impute data for sector column**

First we check if there are rows where the sector is missing but the company name is not missing. Then we will search the dataframe for rows with the same company name with the sector value. We will then put in the sector values for those rows where the sector is missing. 

In [8]:
# Visualise which company has the most missing values. 
# Group by 'company_name' and count missing 'sector' values
missing_sector_counts = new_df.groupby('company_name')['sector'].apply(lambda x: x.isnull().sum()).reset_index(name='missing_sector_count')

# Sort in descending order based on the count of missing values
missing_sector_counts = missing_sector_counts.sort_values(by='missing_sector_count', ascending=False)

# Remove rows where the number of missing_sector_count is 0
missing_sector_counts = missing_sector_counts[missing_sector_counts['missing_sector_count'] > 0]
missing_sector_counts.head(12)

,company_name,missing_sector_count
263,CaterSpot,172
1305,Singapore University of Technology and Design ...,86
264,Catersmith,83
329,Compass Group,54
674,Institute of Singapore Chartered Accountants (...,43
93,Amazon,38
146,Aureus Group Pte. Ltd.,36
1498,VISIONNAIRE EVENTS,32
512,FoodLine,24
330,Compass Group Singapore Pte Ltd.,23


**Impute sector based on campany name and ID**

In [9]:
filtered_df = new_df[new_df['company_name'].str.contains('Institute of Singapore Chartered Accountant', case=False, na=False)]
print(len(filtered_df))
filtered_df

52


,event_id,event_state,sales_type,sector,company_name,customer_id,cost_per_pax,pax,address,postal_code,special_instructions,pickup_date_time,serving_date_time,revenue
5857,G1033,PUBLISHED,outbound,NaN,Institute of Singapore Chartered Accountants (...,49733,14.00,16,"60 Cecil Street, ISCA House",49709,"No carpark for this building, please take note!",2022-08-11T11:15:00,2022-08-11T12:15:00,254.00
5858,G1034,PUBLISHED,outbound,NaN,Institute of Singapore Chartered Accountants (...,49733,14.00,16,"60 Cecil Street, ISCA House",49709,"No carpark for this building, please take note!",2022-08-12T11:15:00,2022-08-12T12:15:00,254.00
6050,G1359,PUBLISHED,repeat,NaN,Institute of Singapore Chartered Accountants (...,49733,14.00,32,"60 Cecil Street, ISCA House",49709,"No carpark for this building, please take note!",2022-08-17T11:00:00,2022-08-17T12:00:00,478.00
6079,G1409,PUBLISHED,repeat,NaN,Institute of Singapore Chartered Accountants (...,49733,14.00,21,"60 Cecil Street, ISCA House",49709,21 x bento + cutlery,2022-08-18T11:00:00,2022-08-18T12:00:00,324.00
6756,G2654,PUBLISHED,repeat,NaN,Institute of Singapore Chartered Accountants (...,49733,0.00,14,"RNN Conference Centre Pte Ltd 137 Cecil St, L...",69537,14 x Bento + Cutlery,2022-09-13T16:30:00,2022-09-13T17:30:00,196.60
7335,G3865,PUBLISHED,repeat,NaN,Institute of Singapore Chartered Accountants (...,49733,0.00,18,"60 Cecil Street, ISCA House",49709,18 x bento + cutlery,2022-10-05T10:30:00,2022-10-05T11:30:00,282.00
7362,G3971,PUBLISHED,repeat,NaN,Institute of Singapore Chartered Accountants (...,49733,0.00,14,"60 Cecil Street, ISCA House",49709,14 x bento + cutlery\n\nTake the side door and...,2022-10-06T10:30:00,2022-10-06T11:30:00,226.04
7416,G4102,PUBLISHED,repeat,NaN,Institute of Singapore Chartered Accountants (...,49733,0.00,18,"60 Cecil Street, ISCA House",49709,18 x bento + cutlery\n\nPlease note our buildi...,2022-10-07T10:30:00,2022-10-07T11:30:00,282.00
7453,G4195,PUBLISHED,repeat,NaN,Institute of Singapore Chartered Accountants (...,49733,0.00,23,"60 Cecil Street, ISCA House",49709,23 x bento + cutleries\n\nPlease note our buil...,2022-10-10T10:30:00,2022-10-10T11:30:00,352.00
7532,G4392,PUBLISHED,repeat,NaN,Institute of Singapore Chartered Accountants (...,49733,14.00,14,"60 Cecil Street, ISCA House",49709,14 x bento + cutlery\n\nPlease note our buildi...,2022-10-11T10:30:00,2022-10-11T11:30:00,226.00


From this we can see that there are some instances where the company_name is not null and there are some instances where the sector is filled up and other instances where the sector is a null value. We will impute the null values with the sector values from the other rows where the sector is not null. 

In [10]:
# Create a separate group for rows where 'company_name' is null
null_company_group = new_df[new_df['company_name'].isnull()]

# Group the DataFrame by 'company_name' (excluding null values)
groups = new_df[new_df['company_name'].notnull()].groupby('company_name', group_keys=True)

# Define a function to impute the sector within each group
def impute_sector(group):
    non_null_sectors = group['sector'].dropna()
    
    if not non_null_sectors.empty:
        first_non_null_sector = non_null_sectors.iloc[0]
        group['sector'].fillna(first_non_null_sector, inplace=True)
    
    return group

# Apply the impute_sector function to each group
imputed_groups = groups.apply(impute_sector)

# Concatenate the null_company_group and imputed_groups back together
imputed_df = pd.concat([null_company_group, imputed_groups])

# Reset the index of the imputed DataFrame
imputed_df.reset_index(drop=True, inplace=True)

# Print the resulting DataFrame
imputed_df

,event_id,event_state,sales_type,sector,company_name,customer_id,cost_per_pax,pax,address,postal_code,special_instructions,pickup_date_time,serving_date_time,revenue
0,E8162,PUBLISHED,inbound,NaN,NaN,45602,18.00,15,"120 King's Rd, Church of St. Ignatius",268172,Customised label: Thank you for making this sp...,2022-02-26T10:45:00,2022-02-26T11:45:00,377.85
1,F2598,PUBLISHED,inbound,NaN,NaN,49671,33.60,5,"123 Teck Whye Lane, #06-850",680123,Please hand over the delivery to a household m...,2022-01-08T18:00:00,2022-01-08T19:00:00,230.67
2,F2672,PUBLISHED,repeat,NaN,NaN,47363,18.00,126,"Amazing Grace Presbyterian Church, 408B Upper ...",466484,252 x Bento + Cutlery \n\n130 x paper bags\n\n...,2022-01-08T10:30:00,2022-01-08T11:30:00,2333.00
3,F3176,PUBLISHED,inbound,NaN,NaN,50203,0.00,10,"910 Hougang Street 91, #06-80",530910,Morning Bundle B and E\n23 x Bentos and Cutleries,2022-01-08T07:30:00,2022-01-08T08:30:00,428.85
4,F3177,PUBLISHED,inbound,NaN,NaN,46587,16.73,18,Block 432 Clementi Ave 3 #06-294,120432,Please take the single lift lobby (beside Poon...,2022-01-08T12:00:00,2022-01-08T13:00:00,321.79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11116,F4360,PUBLISHED,repeat,NaN,wyeth nutrition,16441,40.91,5,Blk 2 Balestier Hill Shopping Centre #04-669,320002,NaN,2022-02-03T11:00:00,2022-02-03T12:00:00,229.55
11117,F4769,PUBLISHED,repeat,NaN,wyeth nutrition,24879,33.60,5,Mt alvernia Hospital 820 Thomson Road #07-63 ...,574623,NaN,2022-02-16T11:15:00,2022-02-16T12:15:00,183.00
11118,F4771,PUBLISHED,repeat,NaN,wyeth nutrition,24879,30.00,10,Mt alvernia Hospital 820 Thomson Road #07-63 ...,574623,deliver togther with F4769,2022-02-16T11:15:00,2022-02-16T12:15:00,383.04
11119,G9673,PUBLISHED,repeat,NaN,wyeth nutrition,24879,0.00,40,"100 Bukit Timah Road, KKH Hospital, Dietetics ...",229899,40 x bento + cutlery,2023-01-05T11:00:00,2023-01-05T12:00:00,579.40


In [11]:
filtered_df = imputed_df[imputed_df['company_name'].str.contains('Institute of Singapore Chartered Accountant', case=False, na=False)]
print(len(filtered_df))
filtered_df

52


,event_id,event_state,sales_type,sector,company_name,customer_id,cost_per_pax,pax,address,postal_code,special_instructions,pickup_date_time,serving_date_time,revenue
6462,G1033,PUBLISHED,outbound,Professional services,Institute of Singapore Chartered Accountants (...,49733,14.00,16,"60 Cecil Street, ISCA House",49709,"No carpark for this building, please take note!",2022-08-11T11:15:00,2022-08-11T12:15:00,254.00
6463,G1034,PUBLISHED,outbound,Professional services,Institute of Singapore Chartered Accountants (...,49733,14.00,16,"60 Cecil Street, ISCA House",49709,"No carpark for this building, please take note!",2022-08-12T11:15:00,2022-08-12T12:15:00,254.00
6464,G1359,PUBLISHED,repeat,Professional services,Institute of Singapore Chartered Accountants (...,49733,14.00,32,"60 Cecil Street, ISCA House",49709,"No carpark for this building, please take note!",2022-08-17T11:00:00,2022-08-17T12:00:00,478.00
6465,G1409,PUBLISHED,repeat,Professional services,Institute of Singapore Chartered Accountants (...,49733,14.00,21,"60 Cecil Street, ISCA House",49709,21 x bento + cutlery,2022-08-18T11:00:00,2022-08-18T12:00:00,324.00
6466,G2654,PUBLISHED,repeat,Professional services,Institute of Singapore Chartered Accountants (...,49733,0.00,14,"RNN Conference Centre Pte Ltd 137 Cecil St, L...",69537,14 x Bento + Cutlery,2022-09-13T16:30:00,2022-09-13T17:30:00,196.60
6467,G3865,PUBLISHED,repeat,Professional services,Institute of Singapore Chartered Accountants (...,49733,0.00,18,"60 Cecil Street, ISCA House",49709,18 x bento + cutlery,2022-10-05T10:30:00,2022-10-05T11:30:00,282.00
6468,G3971,PUBLISHED,repeat,Professional services,Institute of Singapore Chartered Accountants (...,49733,0.00,14,"60 Cecil Street, ISCA House",49709,14 x bento + cutlery\n\nTake the side door and...,2022-10-06T10:30:00,2022-10-06T11:30:00,226.04
6469,G4102,PUBLISHED,repeat,Professional services,Institute of Singapore Chartered Accountants (...,49733,0.00,18,"60 Cecil Street, ISCA House",49709,18 x bento + cutlery\n\nPlease note our buildi...,2022-10-07T10:30:00,2022-10-07T11:30:00,282.00
6470,G4195,PUBLISHED,repeat,Professional services,Institute of Singapore Chartered Accountants (...,49733,0.00,23,"60 Cecil Street, ISCA House",49709,23 x bento + cutleries\n\nPlease note our buil...,2022-10-10T10:30:00,2022-10-10T11:30:00,352.00
6471,G4392,PUBLISHED,repeat,Professional services,Institute of Singapore Chartered Accountants (...,49733,14.00,14,"60 Cecil Street, ISCA House",49709,14 x bento + cutlery\n\nPlease note our buildi...,2022-10-11T10:30:00,2022-10-11T11:30:00,226.00


From this specific exmaple, we see that the rows with the company name as 'Institute of Singapore Chartered Accountants (ISCA)' and have null values for the sector had their sectors filled up with 'Professional services'.

In [12]:
# Get the data types by column
data_types_by_column = imputed_df.dtypes

# Get the number of null values by column
null_values_by_column = imputed_df.isnull().sum()

# Get the percentage of null values by column
percentage_of_null_values = round((imputed_df.isnull().sum() / len(imputed_df)) * 100)

# Combine the data types and null values into a DataFrame
data_info = pd.DataFrame({'Data Type': data_types_by_column, 'Null Values': null_values_by_column, 'Percentage of null values(%)': percentage_of_null_values})

# Print the result
data_info

,Data Type,Null Values,Percentage of null values(%)
event_id,object,0,0.0
event_state,object,0,0.0
sales_type,object,0,0.0
sector,object,4284,39.0
company_name,object,2596,23.0
customer_id,int64,0,0.0
cost_per_pax,float64,0,0.0
pax,int64,0,0.0
address,object,0,0.0
postal_code,int64,0,0.0


Previously, we had 4453 missing values for the sector column and now we have 4284 missing values in the sector column. We managed to impute 169 values. It is not a lot, so we will impute some of the data manually for the top 10 companies which have missing values. 

**Manually impute the sector for the top 10 missing sectors but have company names**

In [13]:
# Find the row where company_name is 'CaterSpot' and set the sector value
imputed_df.loc[imputed_df['company_name'] == 'CaterSpot', 'sector'] = 'Catering service'

# Find the row where company_name is 'Singapore University of Technology and Design (SUTD)' and set the sector value
imputed_df.loc[imputed_df['company_name'] == 'Singapore University of Technology and Design (SUTD)', 'sector'] = 'Education'

# Find the row where company_name is 'Catersmith' and set the sector value
imputed_df.loc[imputed_df['company_name'] == 'Catersmith', 'sector'] = 'Catering service'

# Find the row where company_name is 'Compass Group' and set the sector value
imputed_df.loc[imputed_df['company_name'] == 'Compass Group', 'sector'] = 'Catering service'

# Find the row where company_name is 'Amazon' and set the sector value
imputed_df.loc[imputed_df['company_name'] == 'Amazon', 'sector'] = 'Technology (software)'

# Find the row where company_name is 'Aureus Group Pte. Ltd.' and set the sector value
imputed_df.loc[imputed_df['company_name'] == 'Aureus Group Pte. Ltd.', 'sector'] = 'Music school'

# Find the row where company_name is 'VISIONNAIRE EVENTS' and set the sector value
imputed_df.loc[imputed_df['company_name'] == 'VISIONNAIRE EVENTS', 'sector'] = 'Events Company'

# Find the row where company_name is 'FoodLine' and set the sector value
imputed_df.loc[imputed_df['company_name'] == 'FoodLine', 'sector'] = 'Catering service'

# Find the row where company_name is 'Compass Group Singapore Pte Ltd.' and set the sector value
imputed_df.loc[imputed_df['company_name'] == 'Compass Group Singapore Pte Ltd.', 'sector'] = 'Catering service'

# Find the row where company_name is 'Grain' and set the sector value
imputed_df.loc[imputed_df['company_name'] == 'Grain', 'sector'] = 'Catering company'


In [14]:
filtered_df = imputed_df[imputed_df['company_name'].str.contains('Grain', case=False, na=False)]
filtered_df.head()

,event_id,event_state,sales_type,sector,company_name,customer_id,cost_per_pax,pax,address,postal_code,special_instructions,pickup_date_time,serving_date_time,revenue
6172,F3714,PUBLISHED,repeat,Catering company,Grain,18528,39.04,5,"5 Burn Road, #05-01 Tee Yih Jia Building,, #05-01",369972,NaN,2022-01-18T11:00:00,2022-01-18T11:00:00,0.00
6173,F3825,PUBLISHED,repeat,Catering company,Grain,2558,39.04,5,"5 Burn Road, #05-01 Tee Yih Jia Building",369972,NaN,2022-01-31T17:45:00,2022-01-31T18:45:00,175.68
6174,F6237,PUBLISHED,repeat,Catering company,Grain,2558,10.50,16,"5 Burn Road, #05-01 Tee Yih Jia Building",369972,NaN,2022-04-13T12:30:00,2022-04-13T13:30:00,0.00
6175,F6262,PUBLISHED,repeat,Catering company,Grain,2558,16.00,10,"5 Burn Road, #05-01 Tee Yih Jia Building",369972,Food tasting + photoshoot,2022-04-14T12:30:00,2022-04-14T13:30:00,0.00
6176,F9677,PUBLISHED,repeat,Catering company,Grain,2558,18.50,10,"5 Burn Road, #05-01 Tee Yih Jia Building",369972,NaN,2022-07-09T11:00:00,2022-07-09T12:00:00,166.50


In [15]:
# Get the data types by column
data_types_by_column = imputed_df.dtypes

# Get the number of null values by column
null_values_by_column = imputed_df.isnull().sum()

# Get the percentage of null values by column
percentage_of_null_values = round((imputed_df.isnull().sum() / len(imputed_df)) * 100)

# Combine the data types and null values into a DataFrame
data_info = pd.DataFrame({'Data Type': data_types_by_column, 'Null Values': null_values_by_column, 'Percentage of null values(%)': percentage_of_null_values})

# Print the result
data_info

,Data Type,Null Values,Percentage of null values(%)
event_id,object,0,0.0
event_state,object,0,0.0
sales_type,object,0,0.0
sector,object,3719,33.0
company_name,object,2596,23.0
customer_id,int64,0,0.0
cost_per_pax,float64,0,0.0
pax,int64,0,0.0
address,object,0,0.0
postal_code,int64,0,0.0


Previously, we had 4284 missing values for the sector column and now we have 3719 missing values in the sector column. We managed to impute 565 values. We stil have a massive number of missing values so we will impute the sector as missing. 

**Lastly, we will impute 'missing' for the rest of the missing values in the sector column**

In [16]:
# Impute 'sector' as 'missing' for rows with null sector values
imputed_df['sector'].fillna('missing', inplace=True)

In [17]:
# Get the data types by column
data_types_by_column = imputed_df.dtypes

# Get the number of null values by column
null_values_by_column = imputed_df.isnull().sum()

# Get the percentage of null values by column
percentage_of_null_values = round((imputed_df.isnull().sum() / len(imputed_df)) * 100)

# Combine the data types and null values into a DataFrame
data_info = pd.DataFrame({'Data Type': data_types_by_column, 'Null Values': null_values_by_column, 'Percentage of null values(%)': percentage_of_null_values})

# Print the result
data_info

,Data Type,Null Values,Percentage of null values(%)
event_id,object,0,0.0
event_state,object,0,0.0
sales_type,object,0,0.0
sector,object,0,0.0
company_name,object,2596,23.0
customer_id,int64,0,0.0
cost_per_pax,float64,0,0.0
pax,int64,0,0.0
address,object,0,0.0
postal_code,int64,0,0.0


**Impute data for company_name column**

In [18]:
# Impute 'sector' as 'missing' for rows with null sector values
imputed_df['company_name'].fillna('missing', inplace=True)

In [19]:
# Get the data types by column
data_types_by_column = imputed_df.dtypes

# Get the number of null values by column
null_values_by_column = imputed_df.isnull().sum()

# Get the percentage of null values by column
percentage_of_null_values = round((imputed_df.isnull().sum() / len(imputed_df)) * 100)

# Combine the data types and null values into a DataFrame
data_info = pd.DataFrame({'Data Type': data_types_by_column, 'Null Values': null_values_by_column, 'Percentage of null values(%)': percentage_of_null_values})

# Print the result
data_info

,Data Type,Null Values,Percentage of null values(%)
event_id,object,0,0.0
event_state,object,0,0.0
sales_type,object,0,0.0
sector,object,0,0.0
company_name,object,0,0.0
customer_id,int64,0,0.0
cost_per_pax,float64,0,0.0
pax,int64,0,0.0
address,object,0,0.0
postal_code,int64,0,0.0


**Impute data for special_instructions column**

In [20]:
# Impute 'sector' as 'missing' for rows with null sector values
imputed_df['special_instructions'].fillna('missing', inplace=True)

In [21]:
# Get the data types by column
data_types_by_column = imputed_df.dtypes

# Get the number of null values by column
null_values_by_column = imputed_df.isnull().sum()

# Get the percentage of null values by column
percentage_of_null_values = round((imputed_df.isnull().sum() / len(imputed_df)) * 100)

# Combine the data types and null values into a DataFrame
data_info = pd.DataFrame({'Data Type': data_types_by_column, 'Null Values': null_values_by_column, 'Percentage of null values(%)': percentage_of_null_values})

# Print the result
data_info

,Data Type,Null Values,Percentage of null values(%)
event_id,object,0,0.0
event_state,object,0,0.0
sales_type,object,0,0.0
sector,object,0,0.0
company_name,object,0,0.0
customer_id,int64,0,0.0
cost_per_pax,float64,0,0.0
pax,int64,0,0.0
address,object,0,0.0
postal_code,int64,0,0.0


Now that all the data has been imputted, we wil sort the dataframe by date and time.

In [22]:
# Sort the dataframe by time
imputed_df = imputed_df.sort_values(by='pickup_date_time', ascending=True)
imputed_df.reset_index(drop=True, inplace=True)
imputed_df.head()

,event_id,event_state,sales_type,sector,company_name,customer_id,cost_per_pax,pax,address,postal_code,special_instructions,pickup_date_time,serving_date_time,revenue
0,F3253,PUBLISHED,inbound,Technology (software),Amazon Web Services Singapore Pte Ltd,50296,16.73,23,"8 Marina View, #23-01 Asia Square Tower 1",18960,call Mona upon upon arrival at level 23,2022-01-06T07:00:00,2022-01-06T08:00:00,399.79
1,F3201,PUBLISHED,repeat,Pharmaceutical,Novartis (Singapore) Pte Ltd,26299,7.43,30,"1E Kent Ridge Rd, National University Cancer I...",119228,"30 x Bento + Cutlery | 39 x ripple cups, lids,...",2022-01-06T07:15:00,2022-01-06T08:15:00,308.13
2,F3259,PUBLISHED,inbound,"Government, Nonprofits & Associations",Ministry of Manpower,50306,16.73,39,"18 Havlock Road, Main Entrance, Ministry of Ma...",59764,missing,2022-01-06T10:30:00,2022-01-06T11:30:00,661.77
3,F3239,PUBLISHED,repeat,Hospitals,KK Women's and Children's Hospital,47121,9.30,50,"100 Bukit Timah Road, Department of Maternal F...",229899,50 x bento + cutlery | \n\nMs. Jasmine Lim – 9...,2022-01-06T10:30:00,2022-01-06T11:30:00,465.00
4,F3254,PUBLISHED,inbound,Technology (software),Amazon Web Services Singapore Pte Ltd,50296,0.00,23,"8 Marina View, #23-01 Asia Square Tower 1",18960,missing,2022-01-06T10:30:00,2022-01-06T11:30:00,15.00


**Get the new postal code to have leading zeros**

Some of the postal codes have 5 digits which is not correct as there is supposed to be a 0 in front. Therefore, we will add the leading zeros to the postal codes.

In [26]:
# Assuming 'df' is your original DataFrame
new_df1 = imputed_df.copy()

# Convert the 'postal_code' column to integer
new_df1['postal_code'] = new_df1['postal_code'].astype(str)

# Define a function to add a leading zero to 5-digit postal codes
def add_leading_zero(p_code):
    if len(p_code) == 5:
        return '0' + p_code
    else:
        return p_code

# Apply the function to the 'postal_code' column in the DataFrame
new_df1['postal_code_new'] = new_df1['postal_code'].apply(add_leading_zero)

# Convert the 'postal_code' column back to integer
new_df1['postal_code'] = new_df1['postal_code'].astype(int)

# Display the sorted DataFrame
new_df1.head()

,event_id,event_state,sales_type,sector,company_name,customer_id,cost_per_pax,pax,address,postal_code,special_instructions,pickup_date_time,serving_date_time,revenue,postal_code_new
0,F3253,PUBLISHED,inbound,Technology (software),Amazon Web Services Singapore Pte Ltd,50296,16.73,23,"8 Marina View, #23-01 Asia Square Tower 1",18960,call Mona upon upon arrival at level 23,2022-01-06T07:00:00,2022-01-06T08:00:00,399.79,018960
1,F3201,PUBLISHED,repeat,Pharmaceutical,Novartis (Singapore) Pte Ltd,26299,7.43,30,"1E Kent Ridge Rd, National University Cancer I...",119228,"30 x Bento + Cutlery | 39 x ripple cups, lids,...",2022-01-06T07:15:00,2022-01-06T08:15:00,308.13,119228
2,F3259,PUBLISHED,inbound,"Government, Nonprofits & Associations",Ministry of Manpower,50306,16.73,39,"18 Havlock Road, Main Entrance, Ministry of Ma...",59764,missing,2022-01-06T10:30:00,2022-01-06T11:30:00,661.77,059764
3,F3239,PUBLISHED,repeat,Hospitals,KK Women's and Children's Hospital,47121,9.30,50,"100 Bukit Timah Road, Department of Maternal F...",229899,50 x bento + cutlery | \n\nMs. Jasmine Lim – 9...,2022-01-06T10:30:00,2022-01-06T11:30:00,465.00,229899
4,F3254,PUBLISHED,inbound,Technology (software),Amazon Web Services Singapore Pte Ltd,50296,0.00,23,"8 Marina View, #23-01 Asia Square Tower 1",18960,missing,2022-01-06T10:30:00,2022-01-06T11:30:00,15.00,018960


With the imputted data and sorted dataframe, we will save the file as a new csv file and move on to exploratory data analysis.

In [27]:
new_df1.to_csv('../data/imputed_data.csv', index=False)